## Note: Has to be run with GPU support

In [ ]:
!pip install torch===1.7.0 torchvision===0.8.1 torchaudio===0.7.0 -f https://download.pytorch.org/whl/torch_stable.html
!pip install --upgrade pip
!pip install transformers==3.5.1
!pip install tokenizers==0.9.3
!pip install xlrd
!pip install tabulate
!pip install simpletransformers

In [4]:
'''basics'''
import os
import sys
sys.path.insert(0, os.path.abspath(os.path.join('../..', 'src')))
sys.setrecursionlimit(20500)

import pandas as pd
import pickle 
import numpy as np

import sklearn

from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
from ipywidgets import IntProgress
#from tabulate import tabulate
import string
import re

'''transformer'''
# from simpletransformers.classification import (MultiLabelClassificationModel, MultiLabelClassificationArgs)
# #from simpletransformers.classification import ClassificationModel, ClassificationArgs
# import torch
# from scipy.special import softmax
# import numpy
# import pandas as pd
# import logging
# import wandb

'transformer'

In [5]:
#df = pd.read_csv('_hot_topics_clean.csv')  

df = pd.read_csv(os.path.abspath(os.path.join('../..', 'data/processed/encoded_labels'))+'/_hot_topics.csv') 

In [16]:
# transform to multi label problem

df_relevant = df[['all_text_clean',
       'food_and_agricultural_commodities_strategy', 'green_recovery',
       'health', 'human_rights', 'leaving_no_one_behind',
       'multi_stakeholder_collaboration', 'nature_based_solution', 'no tag',
       'plastic', 'poverty_reduction', 'public_private_partnership', 'sids',
       'south_south_cooperation', 'structural_system_transformation']]

cols = df_relevant.columns
label_cols = list(cols[1:])
num_labels = len(label_cols)

df_relevant['labels'] = list(df_relevant[label_cols].values)
df_alpha = df_relevant[['all_text_clean', 'labels']]

<ipython-input-16-f28a2c8caafe>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_relevant['labels'] = list(df_relevant[label_cols].values)


In [ ]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
"""No Hyperparameter tuning - simple model training"""


model = MultiLabelClassificationModel('longformer', 
                                      'allenai/longformer-base-4096', 
                                      num_labels=num_labels, 
                                      #pos_weight = weights,
                                      args={'overwrite_output_dir': True, 'max_seq_length': 4096, "save_eval_checkpoints": False, "train_batch_size": 1, 
                                            "fp16": True, "num_train_epochs": 5})

# Train the model
model.train_model(train_df, eval_df=eval_df)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df)